In [1]:
import sys
!conda install --yes --prefix {sys.prefix} pip
!{sys.executable} -m pip install krippendorff

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - pip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py39hecd8cb5_0        14.4 MB
    ------------------------------------------------------------
                                           Total:        14.4 MB

The following packages will be UPDATED:

  conda                               4.10.3-py39hecd8cb5_0 --> 4.11.0-py39hecd8cb5_0



conda-4.11.0         | 14.4 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
import numpy as np 
import pandas as pd 
#!/usr/bin/env python
from pandas import DataFrame
import krippendorff

''' 
for disagreement of errors
'ann' should be a df with three columns: 1.coder; 2.code; 3.id. 
'''
def create_df(ann):  
    dflist =[]
    codelist = list(ann.code.unique())
    unique_code = ann.code.unique() # len = 181
    for j in range(0,len(unique_code)):  # col
        df = pd.DataFrame(0,columns = list(ann.id.unique()), index =list(ann.coder.unique()))
        for i in ann.index:
            if ann.iloc[i][1] ==unique_code[j]:  # j=0
                for coder in list(ann.coder.unique()):
                    if coder == ann.iloc[i][0]:
                        for error in list(ann.id.unique()):
                            if error == ann.iloc[i][2]:
                                df.loc[coder][error]+=1
        alist = df.values.tolist()
        dflist.append(alist)
    return dflist,codelist

''' 'ann' should be a df with three columns: 1.coder; 2.id; 3.code. '''
def create_single_df(ann):  
    dflist =[]
    idlist=list(ann.id.unique())
    #unique_id = ann.descr_id.unique() # len = 178
    unique_id = ann.id.unique() # len = 181
    for j in range(0,len(unique_id)):  # col
        df = pd.DataFrame(0,columns = list(ann.code.unique()), index =list(ann.coder.unique()))
        for i in ann.index:
            if ann.iloc[i][1] ==unique_id[j]:  # j=0
                for coder in list(ann.coder.unique()):
                    if coder == ann.iloc[i][0]:
                        for error in list(ann.code.unique()):
                            if error == ann.iloc[i][2]:
                                df.loc[coder][error]+=1
        alist = df.values.tolist()
        dflist.append(alist)
    return dflist,idlist

def total(dflist):
    total = 0
    for j in dflist:
        for i in j:
            total +=sum(i)
    return total

def nestedlist_to_str(dflist):
    for i in range(0,len(dflist)):
        for j in range(0,len(dflist[i])):
            dflist[i][j] =  " ".join(str(e) for e in dflist[i][j])
    return dflist

'''krippendorff score function'''
def Krippendroffis_alpha(ls):
    l=[]
    for i in range(0,len(ls)):
        reliability_data_str = (ls[i])
        reliability_data = [[np.nan if v == '*' else int(v) for v in coder.split()] for coder in reliability_data_str]
        #print("Krippendorff's alpha for nominal metric",i,":", krippendorff.alpha(reliability_data=reliability_data,
        #                                                                      level_of_measurement='nominal'))
        l.append(krippendorff.alpha(reliability_data=reliability_data,level_of_measurement='nominal'))
    return l

In [3]:
import os
for dirname, _, filenames in os.walk('/Users/tianyisun/Desktop/NLQbatches/batch7'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch7.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch7_sql-category_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/batch7_sql-type_Krippendroffis_alpha.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/annotation_feedback_batch7.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/.DS_Store
/Users/tianyisun/Desktop/NLQbatches/batch7/qporepair.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/batch7_annotationtask_sql_errorcategory.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/nz_cprepair2.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/batch7_category_Krippendroffis_alpha.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch7_category_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/batch7_annotationtask_sql_type.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch7_sql-type_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/cprepair2.csv
/Users/tianyisun/Desktop/NLQbatches/batch7/cprep

In [4]:
'''Krippendroffis alpha for error_category'''
errorcategory= pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch4567_anntask_sql_errorcategory.csv')
errorcategory = errorcategory[['username','sql_id','code']]
errorcategory = errorcategory.rename(columns ={'username':'coder'}) 
errorcategory = errorcategory.rename(columns ={'sql_id':'id'}) 
#errorcategory = errorcategory.rename(columns ={'error_classification':'code'}) 
disagreement = errorcategory[['coder','code','id']]
len(errorcategory.id.unique())

1098

In [5]:
''' category Krippendroffis alpha'''
disagreement_dflist, disagreement_idlist = create_df(disagreement)
#print('creat df', disagreement_dflist)
print(len(disagreement_dflist)) # 40 
print(len(disagreement_dflist[0])) # 4
print(total(disagreement_dflist)) # 287
disagreement_ls = nestedlist_to_str(disagreement_dflist)
disagreement_l = Krippendroffis_alpha(disagreement_ls)
#disagreement_l[9] = 0.7550345781466113
print(len(disagreement_l))
disagreement_ave_Krippendroffis_alpha = sum(disagreement_l)/len(disagreement_l)
print(disagreement_ave_Krippendroffis_alpha)  
df = pd.DataFrame([disagreement_idlist,disagreement_l])
df = df.T
df.columns = ['category','Krippendroffis_alpha'] 
result = pd.DataFrame([['average',disagreement_ave_Krippendroffis_alpha]], columns=['category','Krippendroffis_alpha'])
df = df.append(result, ignore_index = True)
df = df.set_index('category')
df

13
2
6325
13
0.8415733320031914


,Krippendroffis_alpha
category,
F,0.779149
D,0.817972
A,0.82571
K,0.788702
C,0.74256
L,0.796083
B,0.661136
N,0.785702
E,0.877084


In [6]:
df.to_csv('batch7/correct_batch4567_category_Krippendroffis_alpha.csv', index = True)

In [7]:
'''Krippendroffis alpha for errorcategory - sql_id Krippendroffis alpha'''
errorcategory_dflist, errorcategory_idlist = create_single_df(errorcategory)
#print('creat single df', disagreement_dflist)
print(len(errorcategory_dflist)) # 178
print(len(errorcategory_dflist[0])) # 4
print(total(errorcategory_dflist)) # 981
errorcategory_ls = nestedlist_to_str(errorcategory_dflist)
errorcategory_l = Krippendroffis_alpha(errorcategory_ls)
errorcategory_ave_Krippendroffis_alpha = sum(errorcategory_l)/len(errorcategory_l)
print(errorcategory_ave_Krippendroffis_alpha) 
df = pd.DataFrame([errorcategory_idlist,errorcategory_l])
df = df.T
df.columns = ['errorcategory','category_Krippendroffis_alpha'] 
df.errorcategory = df.errorcategory.astype(int)
result = pd.DataFrame([['average',errorcategory_ave_Krippendroffis_alpha]], columns=['errorcategory','category_Krippendroffis_alpha'])
df = df.append(result, ignore_index = True)
df = df.set_index('errorcategory')
df

1098
2
6325
0.8250118481773835


,category_Krippendroffis_alpha
errorcategory,
1024,1.000000
1025,0.761905
1026,1.000000
689,0.583333
1027,0.583333
...,...
396,1.000000
397,1.000000
512,0.000000


In [8]:
df.to_csv('batch7/correct_batch4567_sql-category_Krippendroffis_alpha.csv', index = True)

----

In [9]:
'''Krippendroffis alpha for error_type - sql_id Krippendroffis alpha'''
errortype= pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch4567_anntask_sql_type.csv')
errortype = errortype[['username','sql_id','code']]
errortype = errortype.rename(columns ={'username':'coder'}) 
errortype = errortype.rename(columns ={'sql_id':'id'}) 
#errortype = errortype.rename(columns ={'error_classification':'code'}) 
len(errorcategory.id.unique())

1098

In [10]:
'''Krippendroffis alpha'''
errortype_dflist, errortype_idlist = create_single_df(errortype)
print(len(errortype_dflist)) # 178
print(len(errortype_dflist[0])) # 4
print(total(errortype_dflist)) # 981
errortype_ls = nestedlist_to_str(errortype_dflist)
errortype_l = Krippendroffis_alpha(errortype_ls)
errortype_ave_Krippendroffis_alpha = sum(errortype_l)/len(errortype_l)
print(errortype_ave_Krippendroffis_alpha) 
df2 = pd.DataFrame([errortype_idlist,errortype_l])
df2 = df2.T
df2.columns = ['id','type_Krippendroffis_alpha'] 
#df2.errortype = df2.errortype.astype(int)
result = pd.DataFrame([['average',errortype_ave_Krippendroffis_alpha]], columns=['id','type_Krippendroffis_alpha'])
df2 = df2.append(result, ignore_index = True)
df2 = df2.set_index('id')
df2

1098
2
5007
0.7039702201465525


,type_Krippendroffis_alpha
id,
1024.0,0.659498
1025.0,0.791209
1026.0,1.000000
689.0,0.659498
1027.0,0.659498
...,...
396.0,1.000000
397.0,1.000000
512.0,-0.010638


In [11]:
df2.to_csv('batch7/correct_batch4567_sql-type_Krippendroffis_alpha.csv', index = True)